In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a Data Frame 
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Kaitangata,NZ,2021-09-22 18:18:29,-46.2817,169.8464,39.78,88,7,4.47
1,1,Beckley,US,2021-09-22 18:18:30,37.7782,-81.1882,76.59,100,90,21.85
2,2,Yellowknife,CA,2021-09-22 18:15:04,62.4560,-114.3525,52.39,69,49,9.10
3,3,Coihaique,CL,2021-09-22 18:18:30,-45.5752,-72.0662,56.01,58,75,2.30
4,4,Rikitea,PF,2021-09-22 18:18:31,-23.1203,-134.9692,73.62,72,0,20.83


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key= g_key)

In [21]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.

fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Heatmap of percent humidity 
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level = 1.5)
heat_layer = gmaps.heatmap_layer(locations, weights= humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
# Heatmap for cloudiness 
locations = city_data_df[["Lat", "Lng"]]
cloudiness = city_data_df["Cloudiness"]
fig = gmaps.figure(center= (30.0, 31.0), zoom_level = 1.5)
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating= False, max_intensity = 300, point_radius= 4)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
# heatmap for wind speed
locations = city_data_df[["Lat", "Lng"]]
winds = city_data_df["Wind Speed"]
fig = gmaps.figure(center= (30.0, 31.0), zoom_level = 1.5)
heat_layer = gmaps.heatmap_layer(locations, weights = winds, dissipating= False, max_intensity = 50, point_radius= 4)

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [29]:
# Ask the customer to add a min & max value 
min_temp = float(input("What is the min temp you would like for your trip?"))
max_temp = float(input("What is the max temp you would like for your trip?"))

What is the min temp you would like for your trip?75
What is the max temp you would like for your trip?90


In [35]:
# Filter Data set to min & max customer gave
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"]<= max_temp) &
                                      (city_data_df["Max Temp"]>= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Beckley,US,2021-09-22 18:18:30,37.7782,-81.1882,76.59,100,90,21.85
7,7,Soyo,AO,2021-09-22 18:18:33,-6.1349,12.3689,75.40,81,72,11.01
8,8,Malindi,KE,2021-09-22 18:18:34,-3.2175,40.1191,76.50,84,6,15.93
9,9,Albany,US,2021-09-22 18:16:56,42.6001,-73.9662,80.80,74,100,1.99
10,10,Pemangkat,ID,2021-09-22 18:18:35,1.1667,108.9667,79.29,84,92,2.64


In [40]:
preferred_cities_df.count()

City_ID       170
City          170
Country       167
Date          170
Lat           170
Lng           170
Max Temp      170
Humidity      170
Cloudiness    170
Wind Speed    170
dtype: int64

In [46]:
preferred_cities_df= preferred_cities_df.dropna()

In [47]:
preferred_cities_df.count()

City_ID       167
City          167
Country       167
Date          167
Lat           167
Lng           167
Max Temp      167
Humidity      167
Cloudiness    167
Wind Speed    167
dtype: int64

In [48]:
#Create DF for hotels in the area
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"]= ""
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Beckley,US,76.59,37.7782,-81.1882,
7,Soyo,AO,75.40,-6.1349,12.3689,
8,Malindi,KE,76.50,-3.2175,40.1191,
9,Albany,US,80.80,42.6001,-73.9662,
10,Pemangkat,ID,79.29,1.1667,108.9667,


In [51]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key }
#Iterate through the Data Frame
for index, row in hotel_df.iterrows():
    # get the lats and lngs
    lat= row["Lat"]
    lng= row["Lng"]
    #add lat & lng to location key for the params 
    params["location"]= f"{lat}, {lng}"
    #Use search temrs "lodging" and lat & lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    #make a request and get the json data 
    hotels = requests.get(base_url, params=params).json()
    # grab the first hotel from the results and store the name 
    try:
        hotel_df.loc[index, "Hotel Name"]= hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [53]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Beckley,US,76.59,37.7782,-81.1882,"Country Inn & Suites by Radisson, Beckley, WV"
7,Soyo,AO,75.40,-6.1349,12.3689,Hotel Mpampa
8,Malindi,KE,76.50,-3.2175,40.1191,African House Resort
9,Albany,US,80.80,42.6001,-73.9662,
10,Pemangkat,ID,79.29,1.1667,108.9667,Ramah Kost KITA
15,Adre,TD,81.03,13.4667,22.2000,
24,Bac Lieu,VN,78.53,9.2850,105.7244,Khách sạn Sài Gòn Bạc Liêu
35,Katsuura,JP,80.71,35.1333,140.3000,Katsuura Hotel Mikazuki
38,Tigzirt,DZ,76.50,36.8886,4.1198,Bloc D tazoumbirt (amar)
39,Lompoc,US,87.21,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"


In [55]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [56]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [57]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))